In [1]:
import open3d as o3d
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from neurals.network import PCN

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


pybullet build time: May 20 2022 19:44:17


In [2]:
num_pcds = 40
pcds = []
dfs = []
for i in range(num_pcds):
    pcds.append(o3d.io.read_point_cloud(f"data/seeds_scale/pointclouds/pose_{i:02}_pcd.ply"))
    dfs.append(np.load(f"data/seeds_scale/pointclouds_df/pose_{i:02}_pcd.npy"))


In [3]:
points_list = []
dfs_list = []
for pcd, df in zip(pcds, dfs):
    for i in range(60):
        points = np.asarray(pcd.points)
        points_index = np.random.choice(len(points), 1024, replace=False)
        points = points[points_index]
        points_list.append(points)
        dfs_list.append(df[points_index])
print(len(points_list), len(dfs_list))

2400 2400


In [4]:
class TestDataset(Dataset):
    def __init__(self, points_list, dfs_list):
        self.pcds = np.asarray(points_list)
        self.dfs = np.asarray(dfs_list)
        self.mapping = []
        for i in range(num_pcds):
            self.mapping += [i] * 60
        self.mapping = np.array(self.mapping).astype(np.int64)

    def __len__(self):
        return len(self.mapping)

    def __getitem__(self, index):
        mapping = self.mapping[index]
        pcd = self.pcds[index]
        df = self.dfs[index]
        ret_pcd = pcd + np.random.normal(size=pcd.shape, scale = 0.005)
        ret_df = df + np.random.normal(size=df.shape, scale=0.005)
        return mapping, ret_pcd, ret_df

dataset = TestDataset(points_list, dfs_list)
(training_dataset, test_dataset) = \
        torch.utils.data.random_split(
            dataset, (2100, 300))

In [6]:
class TestNN(nn.Module):
    def __init__(self):
        super(TestNN, self).__init__()
        self.pcn = PCN(4,latent_dim=20)
        #self.pcn = PointNet2(feat_dim=0)
        self.fc1 = nn.Linear(20, 64)
        self.out = nn.Linear(64, num_pcds)

    def forward(self, pcd, df):
        df = df.view([df.shape[0], df.shape[1],1])
        pcd = torch.cat([pcd, df], axis=2)
        latent = self.pcn(pcd)
        x = self.fc1(latent).relu()
        x = torch.nn.functional.log_softmax(self.out(x), dim=1)
        return x
net = TestNN().cuda()

In [7]:
optim = torch.optim.Adam(net.parameters(), lr=1e-3)
loss_fn = torch.nn.NLLLoss()
dataloader = DataLoader(training_dataset, batch_size=64)

In [8]:
for i in range(500):
    batch_cnt = 0
    total_loss = 0
    for label, pointcloud, df in dataloader:
        optim.zero_grad()
        label = label.cuda()
        pointcloud = pointcloud.cuda().float()
        df = df.cuda().float()
        pred = net(pointcloud, df)
        loss = loss_fn(pred, label)
        loss.backward()
        optim.step()
        batch_cnt += 1
        total_loss += float(loss)
    print(f"Total Loss: {total_loss/batch_cnt}")

Total Loss: 3.693950320735122
Total Loss: 3.6900267528765127
Total Loss: 3.6841722112713438
Total Loss: 3.5747226079305015
Total Loss: 3.009791547601873
Total Loss: 2.4957367651390308
Total Loss: 2.206693613167965
Total Loss: 2.011864748868075
Total Loss: 1.8696260994130915
Total Loss: 1.769778605663415
Total Loss: 1.6740519458597356
Total Loss: 1.5657452525514546
Total Loss: 1.4723020466891201
Total Loss: 1.3520053588982783
Total Loss: 1.2541904521710945
Total Loss: 1.1624847686651982
Total Loss: 1.055526650313175
Total Loss: 0.9744395234368064
Total Loss: 0.9025665124257406
Total Loss: 0.8367806293747642
Total Loss: 0.7829410415707212
Total Loss: 0.7229352575359922
Total Loss: 0.733105896097241
Total Loss: 0.6748475929101309
Total Loss: 0.6430729130903879
Total Loss: 0.6409707548040332
Total Loss: 0.5595250915397297
Total Loss: 0.5181558818528147
Total Loss: 0.48966885516137787
Total Loss: 0.4768274589018388
Total Loss: 0.4483107364538944
Total Loss: 0.41718383810736914
Total Loss: 0

In [9]:
dataloader = DataLoader(test_dataset, batch_size=1)
batch_cnt = 0
total_loss = 0
net.eval()
labels = []
for label, pointcloud, df in dataloader:
    label = label.cuda()
    pointcloud = pointcloud.cuda().float()
    df = df.cuda().float()
    pred = net(pointcloud, df)
    labels.append(label)
    loss = loss_fn(pred, label)
    batch_cnt += 1
    total_loss += float(loss)
print(f"Total Test Loss: {total_loss/batch_cnt}")

Total Test Loss: 0.05825124947567583


In [10]:
torch.save(net.pcn.state_dict(), "neurals/pcn_model/pcn_scale_20.pth")

: 

In [19]:
o3d.visualization.draw_geometries([o3d.geometry.PointCloud(o3d.utility.Vector3dVector(training_dataset[10][1]))])

In [15]:
test_dataset[10][1]

array([[-0.04219856, -0.01532705,  0.03520654],
       [-0.03941568,  0.10910279, -0.05009749],
       [ 0.03348964, -0.15582641,  0.04298963],
       ...,
       [-0.20387647, -0.08532091,  0.06352441],
       [-0.06860191,  0.14903795,  0.05420021],
       [ 0.10703852, -0.02935053,  0.06121993]])